In [32]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import copy
import multiprocessing

In [33]:
#energy calculation of a column (row.T)
def energy(image):
  kernel = np.array([[-1, -2, -1],
                    [0, 0, 0],
                    [-1, -2, -1]])
  energy = cv.filter2D(image, -1, kernel)
  return energy

In [34]:
#remove a seam (row.T)
def remove_seam(image, indexes):
  return np.delete(image, indexes, axis=1)

In [35]:
# sequential Dynamic programming
import numpy as np

def min_index(p_l, p, p_r):
    if p_l <= p and p_l <= p_r:
        return -1
    elif p <= p_l and p <= p_r:
        return 0
    else:
        return 1

def sequential_seam_find(image):
    energy_matrix = energy(image)
    rows, cols = energy_matrix.shape

    # Initialize val only for the valid range (excluding padding)
    val = [energy_matrix[rows - 2][j] for j in range(1, cols - 1)]
    indexes = [[j] for j in range(1, cols - 1)]  # Store indices for each valid column

    for i in range(rows - 3, 0, -1):  # Avoid top and bottom padding
        new_val = [0] * (cols - 2)  # Store updated energy values for valid columns
        new_indexes = [[] for _ in range(cols - 2)]  # Store updated paths

        for j in range(1, cols - 1):  # Avoid left and right padding
            # Check boundaries before accessing neighbors
            p_l = energy_matrix[i][j - 1] if j - 1 >= 1 else float('inf')
            p = energy_matrix[i][j]
            p_r = energy_matrix[i][j + 1] if j + 1 < cols - 1 else float('inf')

            index_offset = min_index(p_l, p, p_r)
            new_val[j - 1] = val[(j - 1) + index_offset] + energy_matrix[i][j]
            new_indexes[j - 1] = indexes[(j - 1) + index_offset] + [j]

        val = new_val
        indexes = new_indexes

    return dict(zip(val, indexes))

In [36]:
# paralle Dynamic programming
def min_index(p_l, p, p_r):
    if p_l <= p and p_l <= p_r:
        return -1
    elif p <= p_l and p <= p_r:
        return 0
    else:
        return 1

def process_seam(args):
    i, j, energy_matrix, val, indexes = args
    rows, cols = energy_matrix.shape

    p_l = energy_matrix[i][j - 1] if j - 1 >= 1 else float('inf')
    p = energy_matrix[i][j]
    p_r = energy_matrix[i][j + 1] if j + 1 < cols - 1 else float('inf')

    index_offset = min_index(p_l, p, p_r)
    new_value = val[j - 1 + index_offset] + p
    new_index = indexes[j - 1 + index_offset] + [j]

    return j - 1, new_value, new_index

def parallel_seam_find(image):
    energy_matrix = energy(image)
    rows, cols = energy_matrix.shape

    val = [energy_matrix[rows - 2][j] for j in range(1, cols - 1)]
    indexes = [[j] for j in range(1, cols - 1)]

    for i in range(rows - 3, 0, -1):
        with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
            results = pool.map(process_seam, [(i, j, energy_matrix, val, indexes) for j in range(1, cols - 1)])

        # Unpack results
        new_val = [0] * (cols - 2)
        new_indexes = [[] for _ in range(cols - 2)]
        for j, new_value, new_index in results:
            new_val[j] = new_value
            new_indexes[j] = new_index

        val, indexes = new_val, new_indexes

    return dict(zip(val, indexes))


In [37]:
# remove seam with min score
def remove_min_seam(image):

    seams = parallel_seam_find(image)
    min_energy = min(seams.keys())
    indexes = seams[min_energy]

    image = remove_seam(image, indexes)

    return image, indexes


In [38]:
# deal with columns
def uni_resize(image, seam_y):
  indexes = []

  for i in range(seam_y):
    image, index = remove_min_seam(image) # will the image be updated after each iteration ?Q
  indexes.append(index)

  return image, indexes

In [39]:
def resize(image, percentage):
  rows, cols = image.shape

  num_seam_x = int(rows * percentage)
  num_seam_y = int(cols * percentage)

  image, row_red_indexes = uni_resize(image.T, num_seam_x)
  image, column_red_indexes = uni_resize(image.T, num_seam_y)

  return image, row_red_indexes, column_red_indexes

In [40]:
def make_red(image, red_indexes):
  image = copy.deepcopy(image)
  shift = 0

  for seam in red_indexes:
      for row, col in enumerate(seam):
          adjusted_col = col + shift
          image[row, adjusted_col] = [255, 0, 0]

          if row > 0:
              if seam[row] > seam[row - 1]:
                  shift += 1
              elif seam[row] < seam[row - 1]:
                  shift -= 1

  return image

In [41]:
image = cv.imread('/content/OIP.jpg')
image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

padded_image = cv.copyMakeBorder(copy.deepcopy(image), 1, 1, 1, 1, cv.BORDER_CONSTANT, value=0)
gray_image = cv.cvtColor(padded_image, cv.COLOR_RGB2GRAY)

row_energy, column_energy = energy(gray_image), energy(gray_image.T)
resized_image, row_red_indexes, column_red_indexes = resize(gray_image, 0.2)
red_image = make_red(make_red(image, row_red_indexes).T, column_red_indexes).T


fig, axes = plt.subplots(2, 5, figsize=(15, 5))
titles = ["Original", "Row Energy", "Column Energy", "Resized", "Deleted Seams"]

for ax, img, title in zip(axes, [image, row_energy, column_energy, resized_image, red_image], titles):
    if len(img.shape) == 2:
        ax.imshow(img, cmap='gray')
    else:
        ax.imshow(img)
    ax.axis("off")

plt.tight_layout()
plt.show()

ValueError: min() arg is an empty sequence